In [1]:
import numpy as np
import numpy.matlib
from scipy.fft import fft, ifft, fft2, ifft2
from scipy import integrate
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import time

# Problem 1 

In [2]:
m = 3
alpha = 0
n = 64

dt = 0.5
T = 25
tspan = np.arange(0,25.5,dt)

L = 10 
x2 = np.linspace(-L/2,L/2,n+1) 
x=x2[0:-1] 
y=x.copy()

B = 1 
D1 = 0.1
D2 = D1 

#Scale our k values to the length-L domain
r1=np.arange(0,n/2,1)
r2=np.arange(-n/2,0,1)
kx = (2*np.pi/L)*np.concatenate((r1,r2))
ky = kx.copy()

In [3]:
#Do the same for the k values
[KX,KY]=np.meshgrid(kx,ky)

#This term shows up in the Laplacian
K = KX**2+KY**2
Kvec = K.reshape(n**2)

In [4]:
#reaction-diffusion system ODE (take the real part at the end?)
def FFT_RD(t, yhat, n):
    
    #Unstack and Convert back to matrix
    u_f = yhat[:4096].reshape(n,n)
    v_f = yhat[4096:].reshape(n,n)

    #Convert back to physical space 
    u = np.real(ifft2(u_f))
    v = np.real(ifft2(v_f))
    
    # #Reshape to n**2
    # u = u.reshape(n**2) #do not do step
    # v = v.reshape(n**2) #do not do step
    
    #defining lambdaA and wA  
    lambda_A = 1 - (u**2 + v**2) #nxn matrix
    w_A = -B*(u**2 + v**2) #nxn matrix
    
    #Convert everything back and return a vector in the Fourier space
    dudt = fft2(lambda_A*u - w_A*v).reshape(n**2,) + (-D1 * Kvec * yhat[:4096]) 
    dvdt = fft2(w_A*u - lambda_A*v).reshape(n**2,) + (-D2 * Kvec * yhat[4096:])
    
    # #reshape
    # dudt = dudt.reshape(n**2,)
    # dvdt = dvdt.reshape(n**2,)
    
    #stack dudt and dvdt 
    rhs = np.hstack([dudt, dvdt])
    
    return rhs

### 1a & 1b -  A1 and A2

In [5]:
X, Y = np.meshgrid(x,y)
m = 3
alpha = 0
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.cos(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.sin(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))

In [6]:
A1 = X.copy()

In [7]:
A2 = u.copy()

### 1c - A3

In [8]:
u0_hat = np.real(fft2(u))
u0hat_vector = u0_hat.reshape(n**2)

v0_hat = np.real(fft2(v))
v0hat_vector = v0_hat.reshape(n**2)

In [9]:
u0_hat.shape

(64, 64)

In [10]:
A3 = u0_hat

### 1d - A4

In [11]:
y0hat = np.hstack([u0hat_vector, v0hat_vector])

In [12]:
y0hat.shape

(8192,)

In [13]:
A4 = np.imag(y0hat).reshape(8192,1)

### 1e - A5

In [14]:
# sol = integrate.solve_ivp(lambda t, y: FFT_RD(t, y), [0, T], y0hat, t_eval = tspan)
sol = integrate.solve_ivp(lambda t, yhat: FFT_RD(t, yhat, n), [0, T], y0hat, t_eval = tspan)

/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [15]:
A5 = np.real(sol.y)

### 1e - A6

In [16]:
A6 = np.imag(sol.y)

### 1f - A7

In [17]:
A7 = sol.y[:4096,4].reshape(4096,1)

### 1g - A8

In [18]:
# z = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
# z.reshape(3,3).T

In [19]:
A8 = A7.reshape(64, 64).T

### 1h - A9

In [20]:
A9 = ifft2(A8)
# y = ifft(sol.y, n=n,axis=0)
#We will plot the modulus squared, which is real
# y = abs(y.T)**2

In [21]:
A9.shape

(64, 64)

### Problem 2 

In [22]:
from cheb import *

In [23]:
N = 30 

[D, x] = cheb(N)
x = x.reshape(N+1)

# Derivative operator squared is second derivative
DD2 = D@D

DD2 = DD2[1:-1,1:-1]
x2 = x[1:-1]
y2 = x2.copy()

[X,Y] = np.meshgrid(x2,y2)
m = 2
alpha = 0

#Initial conditions
u = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.cos(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))
v = (np.tanh(np.sqrt(X**2+Y**2))-alpha)*np.sin(m*np.angle(X+1j*Y) - np.sqrt(X**2+Y**2))

In [24]:
u0_vec = u.reshape((N-1)**2)
v0_vec = v.reshape((N-1)**2)

### 2a - A10

In [25]:
I = np.eye(len(DD2))

Lap = np.kron(DD2, I) + np.kron(I,DD2)

In [26]:
A10 = Lap.copy()

### 2b - A11 

In [27]:
A11 = Y.copy()

### 2c - A12

In [28]:
A12 = v.copy()

### 2d - A13

In [29]:
y0vec = np.hstack([u0_vec, v0_vec])

In [30]:
A13 = y0vec.reshape(1682,1)

### 2e - A14

In [31]:
#reaction-diffusion system ODE
def Chebyshev_RD(t, y, Lap):
    
    #define u and v from y 
    u = y[:841]
    v = y[841:]
        
    #defining lambdaA and wA  
    lambda_A = 1 - (u**2 + v**2)
    w_A = -B*(u**2 + v**2)
    
    dudt = lambda_A*u - w_A*v + D1*(Lap@u) 
    dvdt = w_A*u - lambda_A*v + D2*(Lap@v)
    
    #reshape dudt and dvdt
    dudt = dudt.reshape(841,)
    dvdt = dvdt.reshape(841,)
    
    #stack dudt and dvdt 
    rhs = np.hstack([dudt, dvdt])
    
    return rhs

In [32]:
tic_gaus = time.time()
sol_cheb = integrate.solve_ivp(lambda t, y: Chebyshev_RD(t, y, Lap), [0, T], y0vec, t_eval = tspan)
toc_gaus = time.time()
print(toc_gaus-tic_gaus)

87.67351984977722


### 2f - A15

In [33]:
A15 = sol_cheb.y[841:,4].reshape(841,1)

### 2g - A16

In [34]:
A15_reshape = A15.reshape(29, 29).T

In [35]:
A15_reshape.shape

(29, 29)

In [36]:
zero = np.zeros(N-1)

In [37]:
A16 = np.vstack([zero,A15_reshape,zero])

In [38]:
zero_cols = np.zeros(N+1).reshape(31,1)

In [39]:
A16 = np.hstack([zero_cols,A16,zero_cols])

In [40]:
A16.shape

(31, 31)